<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Citation network of drafts of Chinese constitution

Hive Plot visualization

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pyveplot

In [9]:
from pyveplot import Hiveplot, Axis, Node
import networkx as nx
import random
import csv

nodes = {}
edges = []

with open('L:/ownCloud/DHProjekte/Fupeng Li/nodes.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

    for row in readCSV:
        nodes[row[0]] = (row[1], row[2], row[3], row[4])

with open('L:/ownCloud/DHProjekte/Fupeng Li/edges_v2.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

    for row in readCSV:
        edges.append((row[0], row[1]))

In [10]:
edges

[('source', 'target'),
 ('a01', 'f13'),
 ('a01', 'f12'),
 ('a02', 't07'),
 ('a02', 't04'),
 ('a02', 'f17'),
 ('a02', 'f15'),
 ('a02', 'f03'),
 ('a02', 'f12'),
 ('a02', 'f12'),
 ('a02', 'f76'),
 ('a02', 'f51'),
 ('a02', 'f51'),
 ('a02', 'f57'),
 ('a02', 'f57'),
 ('a02', 'f04'),
 ('a02', 'f04'),
 ('a02', 'f72'),
 ('a02', 'f11'),
 ('a02', 'f70'),
 ('a02', 'f86'),
 ('a02', 'f01'),
 ('a02', 'f67'),
 ('a02', 'f43'),
 ('a02', 'f55'),
 ('a02', 'f58'),
 ('a02', 'f05'),
 ('a02', 'f06'),
 ('a02', 'f39'),
 ('a02', 'f61'),
 ('a02', 'f59'),
 ('a02', 'f07'),
 ('a02', 'f87'),
 ('a02', 'f45'),
 ('a02', 'f09'),
 ('a02', 'f31'),
 ('a02', 'f35'),
 ('a02', 'f64'),
 ('a03', 'f03'),
 ('a03', 'f12'),
 ('a03', 'f12'),
 ('a03', 'f76'),
 ('a03', 'f51'),
 ('a03', 'f51'),
 ('a03', 'f57'),
 ('a03', 'f04'),
 ('a03', 'f72'),
 ('a03', 'f11'),
 ('a03', 'f70'),
 ('a03', 'f86'),
 ('a03', 'f01'),
 ('a03', 'f67'),
 ('a03', 'f43'),
 ('a03', 'f18'),
 ('a03', 'f19'),
 ('a03', 'f38'),
 ('a03', 'f56'),
 ('a03', 'f39'),
 ('a03',

In [ ]:
c = ['#e41a1c', '#377eb8', '#4daf4a',
     '#984ea3', '#ff7f00', '#ffff33',
     '#a65628', '#f781bf', '#999999',]


# create hiveplot object
h = Hiveplot('hiveplot.svg')

# create three axes, spaced at 120 degrees from each other
h.axes = [Axis(start=20, angle=0,
               stroke=random.choice(c), stroke_width=1.1),
          Axis(start=20, angle=120,
               stroke=random.choice(c), stroke_width=1.1),
          Axis(start=20, angle=120 + 120,
               stroke=random.choice(c), stroke_width=1.1)
          ]

# create a random Barabasi-Albert network
g = nx.barabasi_albert_graph(100, 2)

# sort nodes by degree
k = list(nx.degree(g))
k.sort(key=lambda tup: tup[1])

# categorize them as high, medium and low degree
hi_deg = [v[0] for v in k if v[1] > 7]
md_deg = [v[0] for v in k if v[1] > 3 and v[1] <= 7]
lo_deg = [v[0] for v in k if v[1] <= 3]

# place these nodes into our three axes
for axis, nodes in zip(h.axes,
                       [hi_deg, md_deg, lo_deg]):
    circle_color = random.choice(c)
    for v in nodes:
        # create node object
        node = Node(radius=g.degree(v),
                    label="node %s k=%s" % (v, g.degree(v)))
        # add it to axis
        axis.add_node(v, node)
        # once it has x, y coordinates, add a circle
        node.add_circle(fill=circle_color, stroke=circle_color,
                        stroke_width=0.1, fill_opacity=0.7)
        if axis.angle < 180:
            orientation = -1
            scale = 0.6
        else:
            orientation = 1
            scale = 0.35
        # also add a label
        node.add_label("node %s k=%s" % (v, g.degree(v)),
                       angle=axis.angle + 90 * orientation,
                       scale=scale)

# iterate through axes, from left to right
for n in range(-1, len(h.axes) - 1):
    curve_color = random.choice(c)
    # draw curves between nodes connected by edges in network
    h.connect_axes(h.axes[n],
                   h.axes[n+1],
                   g.edges,
                   stroke_width=0.5,
                   stroke=curve_color)

# save output
h.save('ba_hiveplot.svg')